In [1]:
import time
import random
import numpy as np
from sklearn.linear_model import LinearRegression
import collections

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 5, 13, 12
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.5, 0
m3, k3 = 0.15, 0 # this is feedback term
m4, k4 = 0.75, 0.0

dg = 0.1
d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 2
errorB = 4
errorC = 3.5
errorX = 0.75
errorY = 0.6

def jitter(x, scale=1):
    return x + np.random.normal(scale=scale)

def dependent(x, m, c, error=1):
    return jitter(m*x + c, scale=error) # mx +c + error


def linear_positive_C_to_B(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        _, _, C = history[1 - d3]
        _, B, _ = history[-1]
        B_ = jitter(B + A*m1 + C*m3 - B*m2 -dg*B + k1 ,  errorB)
        _, B, _ = history[1 - d2]
        _, _, C = history[-1]
        C_ = jitter(C + B*m2 - dg*C + k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1 ,  errorB)
        _, B, _ = history[1 - d2]
        A, _, _ = history[1 - d1]
        _, _, C = history[-1]
        C_ = jitter(A*m3+ B*m2+ -dg*C + k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def simulations_data(pathway, n=1000):
    random.seed(time.time())
    return np.array([pathway() for i in range(n)])


def regress(X,Y):
    model = LinearRegression()
    mXY = model.fit(X.reshape(-1,1), Y)
    r_sqr = mXY.score(X.reshape(-1,1), Y)
    residual = Y - model.predict(X.reshape(-1, 1))
    return mXY.intercept_, mXY.coef_[0], r_sqr, residual


def get_slope_intercept(model):
    return model._slopt, model._intercept

def compute_regresssion(ABC):
    A,B,C = ABC.transpose()
    RAB = regress(A, B)
    RBC = regress(B, C)
    RAC = regress(A, C)
    corrE = np.corrcoef(np.array([RAB[3], RBC[3]]))
    corrE_BA_C = np.corrcoef(np.array([np.square(RAB[3]), C]))


    #print(RAB[1]*RBC[1]-RAC[1]) ## better to look at distribution of this error..it should come with center as 0
    return {"kAB": RAB[0], "kBC":RBC[0], "kAC":RAC[0],
            "mAB":RAB[1], "mBC":RBC[1], "mAC":RAC[1],
            "r_sqrAB":RAB[2], "r_sqrBC":RBC[2], "r_sqrAC":RAC[2],
            "r_E":corrE[0,1],
            "r_E_BA_C":corrE_BA_C[0,1],
            "n": len(A)}

def compute_correlation(ABC):
    corr = np.corrcoef(ABC.transpose())
    rAB, rBC, rAC = corr[0,1], corr[1,2], corr[0,2]
    #print(rAB**2*rBC**2-rAC**2) ## better to look at distribution of this error..it should come with center as 0
                                ## or see correlation between these two quantities should be 1 and if we regress ,
                                ###       it should have slope 1
    return {"rAB":rAB, "rBC":rBC, "rAC":rAC}


def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        ABC = simulations_data(linear_positive_C_to_B, samplesize) # you can change pathway here
        r = compute_regresssion(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [6]:
def compute_confidence_interval(r, n):
    def boundary(zeta):
        return ((np.exp(2*zeta))-1)/((np.exp(2*zeta))+1)

    z = 0.5*np.log(((1-r)/(1+r)))
    zetal = z-1.96*np.sqrt(1/(n-3))
    rl = boundary(zetal)
    zetau = z+1.96*np.sqrt(1/(n-3))
    ru = boundary(zetau)
    return rl, ru

def confidence_status(L, U, v):
    l = v.where(v < L, "less")
    l.where(v > U, "more", inplace=True)
    w = l.where((v>= L) & (v <= U), "within")
    return w

def confidence_status_(r, n):
    L, U = compute_confidence_interval(r, n)
    return confidence_status(L, U, r)


def add_confidence_stats(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    L,U = compute_confidence_interval(d.rBC**2, d['n'])
    d['confidence_residual_corr'] = confidence_status(L, U, d.r_E)
    L,U = compute_confidence_interval(d.r_E, d['n'])#???
    d['confidence_corrected_bc_corr'] = confidence_status(L, U, d.r_E_BA_C**2)

def confidence_graphs(d):
    confidence = alt.Chart(d, title=f"Correlation confidence{(d['confidence']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_res_corr = alt.Chart(d, title=f"residual corr {(d['confidence_residual_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_residual_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_corrected_bc_corr = alt.Chart(d, title=f"corrected bc corr {(d['confidence_corrected_bc_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_corrected_bc_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    return alt.vconcat(confidence,confidence_res_corr, confidence_corrected_bc_corr)

add_confidence_stats(d)
confidence_graphs(d)

alt.VConcatChart(...)

In [8]:
def stats_graphs(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    slope_histogram = alt.Chart(d).mark_bar().encode(
        x=alt.X('mAB*mBC-mAC:Q', bin=True),
        y='count()').properties(title="slope diff histogram")

    bincount = 100
    ticks = 10
    correlation_graph = alt.Chart(d).mark_bar().encode(
        x=alt.X('rAB2*rBC2-rAC2:Q',bin=True,
                   axis=alt.Axis(
                    tickCount=ticks,
                    grid=False)),
        y='count()').properties(
            title="Correlation")
    residual_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X('r_E:Q', bin=True),
        y='count()').properties(title="Correlation of residuals")
    corrected_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X("r_E_BA_C2-rBC2:Q", bin=True),
        y='count()').properties(
        title="Corrected Correlation")

    return alt.vconcat(slope_histogram,correlation_graph,residual_correlation, corrected_correlation)

stats_graphs(d)

alt.VConcatChart(...)

In [3]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [4]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,r_E_BA_C,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence
0,2522.130779,195.278805,27522.378977,25.668318,10.788401,274.123625,0.520244,0.999830,0.509703,0.582021,0.046962,500,0.721280,0.999915,0.713935,0.010453,-0.997625,2.796486,within
1,2528.060407,175.693073,27609.477030,24.757963,10.794514,263.710696,0.457296,0.999771,0.445161,0.542853,0.025942,500,0.676237,0.999885,0.667204,0.012030,-0.999098,3.539490,within
2,2602.679685,131.138249,28403.293219,22.738183,10.806603,242.146658,0.414045,0.999772,0.401990,0.520889,-0.022764,500,0.643463,0.999886,0.634027,0.011960,-0.999253,3.575857,within
3,2572.820825,151.048848,28083.049217,24.392372,10.804094,260.053479,0.431118,0.999805,0.419713,0.540717,-0.016790,500,0.656596,0.999902,0.647852,0.011321,-0.999523,3.484000,within
4,2549.644229,184.569849,27826.728817,23.889349,10.792324,254.617579,0.472838,0.999790,0.461062,0.558738,-0.003575,500,0.687632,0.999895,0.679016,0.011677,-0.999777,3.204010,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2408.436351,158.592978,26292.089928,26.483416,10.798526,282.750850,0.488798,0.999810,0.477725,0.560997,-0.037566,500,0.699141,0.999905,0.691177,0.010980,-0.998399,3.231023,within
96,2574.425151,165.846869,28098.789374,23.575460,10.798123,251.115020,0.431877,0.999790,0.420144,0.536695,-0.019536,500,0.657174,0.999895,0.648185,0.011643,-0.999408,3.455688,within
97,2593.693039,171.809361,28311.534959,23.177010,10.794991,246.691258,0.435934,0.999771,0.423711,0.537530,-0.065482,500,0.660253,0.999886,0.650931,0.012123,-0.995483,3.504352,within
98,2496.547741,157.341588,27252.259053,26.160557,10.801837,279.379124,0.474097,0.999819,0.463327,0.551191,0.023058,500,0.688547,0.999910,0.680682,0.010685,-0.999288,3.202938,within


In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,r_E_BA_C,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence
0,2522.130779,195.278805,27522.378977,25.668318,10.788401,274.123625,0.520244,0.999830,0.509703,0.582021,0.046962,500,0.721280,0.999915,0.713935,0.010453,-0.997625,2.796486,within
1,2528.060407,175.693073,27609.477030,24.757963,10.794514,263.710696,0.457296,0.999771,0.445161,0.542853,0.025942,500,0.676237,0.999885,0.667204,0.012030,-0.999098,3.539490,within
2,2602.679685,131.138249,28403.293219,22.738183,10.806603,242.146658,0.414045,0.999772,0.401990,0.520889,-0.022764,500,0.643463,0.999886,0.634027,0.011960,-0.999253,3.575857,within
3,2572.820825,151.048848,28083.049217,24.392372,10.804094,260.053479,0.431118,0.999805,0.419713,0.540717,-0.016790,500,0.656596,0.999902,0.647852,0.011321,-0.999523,3.484000,within
4,2549.644229,184.569849,27826.728817,23.889349,10.792324,254.617579,0.472838,0.999790,0.461062,0.558738,-0.003575,500,0.687632,0.999895,0.679016,0.011677,-0.999777,3.204010,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2408.436351,158.592978,26292.089928,26.483416,10.798526,282.750850,0.488798,0.999810,0.477725,0.560997,-0.037566,500,0.699141,0.999905,0.691177,0.010980,-0.998399,3.231023,within
96,2574.425151,165.846869,28098.789374,23.575460,10.798123,251.115020,0.431877,0.999790,0.420144,0.536695,-0.019536,500,0.657174,0.999895,0.648185,0.011643,-0.999408,3.455688,within
97,2593.693039,171.809361,28311.534959,23.177010,10.794991,246.691258,0.435934,0.999771,0.423711,0.537530,-0.065482,500,0.660253,0.999886,0.650931,0.012123,-0.995483,3.504352,within
98,2496.547741,157.341588,27252.259053,26.160557,10.801837,279.379124,0.474097,0.999819,0.463327,0.551191,0.023058,500,0.688547,0.999910,0.680682,0.010685,-0.999288,3.202938,within
